<a href="https://colab.research.google.com/github/davgarveg/Project_Data_Science_-_AI/blob/main/OpenMeteo_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# Instalación de las librerías necesarias

!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
!pip install psycopg2 pandas sqlalchemy

In [18]:
# Importación de las librerías necesarias

import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from sqlalchemy import create_engine

# Configuración de la API de Open-Meteo con caché

cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Aseguramos de que todas las variables meteorológicas que queremos estén enumeradas.
# Tenemos cuidado del órden de las variables para asignarlas correctamente más adelante.

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 36.8381,
	"longitude": -2.4597,
	"start_date": "2022-07-18",
	"end_date": "2025-01-23",
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation"],
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum"],
  "timeformat": "unixtime",
	"timezone": "auto"
}
responses = openmeteo.weather_api(url, params=params)

# Primera ubicación. Utilizamos un bucle "for" para diferentes ubicaciones o modelos meteorológicos.

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Obtenemos datos por hora ya que puede ser interesante al tener una base de datos con datos temporales recogidos por horas también.
# Tenemos en cuenta el orden de las variables de nuevo.

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

# Obtenemos datos por días ya que también me sirven para tener una visualización de los mismos más claras y pueden ayudarme a ajustar el modelo más adelante.
# Una vez más tenemos en cuenta el órden de las variables.

daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 36.8717041015625°N -2.368408203125°E
Elevation 20.0 m asl
Timezone b'Europe/Madrid' b'GMT+1'
Timezone difference to GMT+0 3600 s
                           date  temperature_2m  relative_humidity_2m  \
0     2022-07-17 23:00:00+00:00       29.559000             31.223679   
1     2022-07-18 00:00:00+00:00       28.508999             31.560743   
2     2022-07-18 01:00:00+00:00       28.459000             31.336349   
3     2022-07-18 02:00:00+00:00       27.758999             32.314323   
4     2022-07-18 03:00:00+00:00       27.108999             32.458969   
...                         ...             ...                   ...   
22099 2025-01-23 18:00:00+00:00       16.309000             69.223198   
22100 2025-01-23 19:00:00+00:00       14.858999             74.208054   
22101 2025-01-23 20:00:00+00:00       14.358999             74.869507   
22102 2025-01-23 21:00:00+00:00       14.408999             73.142830   
22103 2025-01-23 22:00:00+00:00       14.108999         

In [20]:
# Configuración de conexión con PostgreSQL ya que tengo mi base de datos recogida allí.

host = "data.hiotera.com"
port = "6432"
database = "hiotera"
user = "hiotera01"
password = "h10Ttt-R1DPq"
sslmode = "require"

# Creamos una conexión a la base de datos

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}?sslmode={sslmode}')

# Guardo el DataFrame en dos tablas distintas, "histórico_clima" con los datos recogidos por horas / "daily_dataframe" con los datos recogidos por días.

nombre_tabla = "histórico_clima"
hourly_dataframe.to_sql(nombre_tabla, engine, if_exists='replace', index=False)

print (f"Datos meteorológicos guardados en la tabla '{hourly_dataframe}' con éxito")

nombre_tabla = "daily_dataframe"
daily_dataframe.to_sql(nombre_tabla, engine, if_exists='replace', index=False)

print (f"Datos meteorológicos guardados en la tabla '{daily_dataframe}' con éxito")



Datos meteorológicos guardados en la tabla '                           date  temperature_2m  relative_humidity_2m  \
0     2022-07-17 23:00:00+00:00       29.559000             31.223679   
1     2022-07-18 00:00:00+00:00       28.508999             31.560743   
2     2022-07-18 01:00:00+00:00       28.459000             31.336349   
3     2022-07-18 02:00:00+00:00       27.758999             32.314323   
4     2022-07-18 03:00:00+00:00       27.108999             32.458969   
...                         ...             ...                   ...   
22099 2025-01-23 18:00:00+00:00       16.309000             69.223198   
22100 2025-01-23 19:00:00+00:00       14.858999             74.208054   
22101 2025-01-23 20:00:00+00:00       14.358999             74.869507   
22102 2025-01-23 21:00:00+00:00       14.408999             73.142830   
22103 2025-01-23 22:00:00+00:00       14.108999             73.088936   

       precipitation  
0                0.0  
1                0.0  
2         

In [21]:
# Exportación de ambas tablas a dos archivos csv.

hourly_dataframe.to_csv('histórico_clima.csv', index=False)
daily_dataframe.to_csv('daily_dataframe.csv', index=False)

print("Datos meteorológicos exportados correctamente")

Datos meteorológicos exportados correctamente
